In [1]:
import torch
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import *
from torch.optim import *

In [2]:
data = pd.read_csv('./data.csv')

In [6]:
data

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [8]:
data.drop(['User ID','Gender','Purchased'],axis=1,inplace=True)

In [12]:
X = torch.from_numpy(np.array(data['Age']))
y = torch.from_numpy(np.array(data['EstimatedSalary']))

In [10]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.linear1 = Linear(1,1000)
        self.linear2 = Linear(1000,2000)
        self.linear3 = Linear(2000,3000)
        self.linear4 = Linear(3000,4000)
        self.linear5 = Linear(4000,1)
    def forward(self,X):
        preds = self.linear1(X)
        preds = self.linear2(preds)
        preds = self.linear3(preds)
        preds = self.linear4(preds)
        preds = self.linear5(preds)
        return preds

In [15]:
device = 'cuda'

In [55]:
class Valid:
    def get_loss(self,criterion,model,X,y):
        preds = model(X.view(-1,1))
        loss = criterion(preds,y)
        return loss.item()
    def accuracy(self,X,y,model):
        model.to('cpu')
        preds = model(X.view(-1,1).float())
        correct = 0
        total = 0
        for idx in range(len(y)):
            if int(y[idx]) == int(preds[idx][0]):
                correct += 1
            total += 1
        acc = round(correct / total,3)
        return acc

In [56]:
model = Model().to(device)
batch_size = 32
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)
epochs = 100
v = Valid()

In [57]:
import wandb
PROJECT_NAME = 'Logistic-Reg'

In [59]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in range(epochs):
    for idx in range(0,len(X),batch_size):
        X_batch = X[idx:idx+batch_size].to(device).view(-1,1)
        y_batch = y[idx:idx+batch_size].to(device).view(-1,1)
        model.to(device)
        preds = model(X_batch.float())
        loss = criterion(preds,y_batch.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({'Loss':loss.item()})
        wandb.log({'Acc':v.accuracy(X,y,model)})
wandb.finish()

Loss,3650750464.0
_runtime,11
_timestamp,1628611878
_step,1
Acc,0.0


Loss,▁
_runtime,▁▁
_timestamp,▁▁
_step,▁█
Acc,▁


wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Loss,777969472.0
_runtime,267
_timestamp,1628612173
_step,2599
Acc,0.0


Loss,▅█▄▂▂▂▁▄▅▂▁▂▂▁▂▂▃▃▂▄▂▂▂▃▂▄▂▂▂▁▁▂▂▄▄▂▂▆█▃
_runtime,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
